In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

# Environment 

In [2]:
#Select Variable 
var = "dlyfrzthw"
#NorESM1-M_rcp45_solidprcptot_monthly

# Run all from here ############

In [10]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

In [11]:
variable85="rcp85_"+var+"_annual"
variable45="rcp45_"+var+"_annual"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_dlyfrzthw_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_dlyfrzthw_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_dlyfrzthw_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_dlyfrzthw_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_dlyfrzthw_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_dlyfrzthw_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_dlyfrzthw_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_dlyfrzthw_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_dlyfrzthw_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_dlyfrzthw_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_dlyfrzthw_annual.nc']

# FOR RCP 45

## Subsetting arrays by periods of time and percentiles with Xclim

In [5]:
dsEns45= ens.create_ensemble(files45)
dsEns45

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    dlyfrzthw    (realization, time, lat, lon) float64 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [6]:
# Mapping data - 30 year means
tmp145 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time145 = tmp145.time[0::window]

ds30yavg45 = tmp145.coarsen(time=window).mean()
ds30yavg45['time'] = time145
perc30yavg45 = ens.ensemble_percentiles(ds30yavg45)
perc30yavg45 # entire grid

# Mapping data

<xarray.Dataset>
Dimensions:        (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * lon            (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat            (lat) float32 66.62331 66.53998 ... 40.12437 40.04104
  * time           (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * realization    (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    dlyfrzthw_p10  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    dlyfrzthw_p50  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    dlyfrzthw_p90  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [7]:
df450 = perc30yavg45.drop('realization').to_dataframe().dropna()

df45 = df45.reset_index()

df45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"df45_anual.feather")

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



In [3]:
df45 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"df45_anual.feather")

## Transform in Celsius and round

In [4]:
df45C = df45.copy()
df45C[var+"_p10"] = round(df45C[var+"_p10"], 2)
df45C[var+"_p50"] = round(df45C[var+"_p50"], 2)
df45C[var+"_p90"] = round(df45C[var+"_p90"], 2)

df45C

,lat,lon,time,dlyfrzthw_p10,dlyfrzthw_p50,dlyfrzthw_p90
0,66.623306,-89.045212,1981-01-01,36.17,38.37,40.97
1,66.623306,-89.045212,2011-01-01,30.93,32.60,36.17
2,66.623306,-89.045212,2041-01-01,22.10,30.23,35.23
3,66.623306,-89.045212,2071-01-01,22.47,28.40,38.50
4,66.623306,-88.961884,1981-01-01,36.17,38.67,41.33
...,...,...,...,...,...,...
304819,40.041039,-74.129143,2071-01-01,61.07,68.63,76.67
304820,40.041039,-74.045807,1981-01-01,87.90,89.20,91.87
304821,40.041039,-74.045807,2011-01-01,76.70,77.87,83.13
304822,40.041039,-74.045807,2041-01-01,65.67,70.67,77.67


## Eliminate 2011, Pivot Table and Create new columns 

In [6]:
Region1i45 = df45C
print (Region1i45)
Region1i245 = Region1i45[Region1i45.time.dt.year!= 2011]
print ("2011 eliminated")
df45f = Region1i245.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df45f.columns = [year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df45f

              lat        lon       time  dlyfrzthw_p10  dlyfrzthw_p50  \
0       66.623306 -89.045212 1981-01-01          36.17          38.37   
1       66.623306 -89.045212 2011-01-01          30.93          32.60   
2       66.623306 -89.045212 2041-01-01          22.10          30.23   
3       66.623306 -89.045212 2071-01-01          22.47          28.40   
4       66.623306 -88.961884 1981-01-01          36.17          38.67   
...           ...        ...        ...            ...            ...   
304819  40.041039 -74.129143 2071-01-01          61.07          68.63   
304820  40.041039 -74.045807 1981-01-01          87.90          89.20   
304821  40.041039 -74.045807 2011-01-01          76.70          77.87   
304822  40.041039 -74.045807 2041-01-01          65.67          70.67   
304823  40.041039 -74.045807 2071-01-01          56.40          63.43   

        dlyfrzthw_p90  
0               40.97  
1               36.17  
2               35.23  
3               38.50  
4  

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212           85.50            67.47            60.50   
          -88.961884           85.67            67.73            60.93   
          -88.878548           85.40            67.63            61.03   
          -88.795219           83.73            66.67            60.40   
          -88.711891           83.20            66.17            59.67   
...                              ...              ...              ...   
66.623306 -61.629639           54.90            43.70            40.43   
          -61.546310           53.63            39.97            38.63   
          -61.462978           54.63            42.30            39.43   
          -61.379650           55.37            42.77            39.90   
          -61.296322           57.10            49.37            45.23   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212           86.80            72.23            68.53   
          -88.961884           86.73            72.83            68.67   
          -88.878548           86.60            72.77            68.77   
          -88.795219           85.30            71.43            67.97   
          -88.711891           85.00            70.93            67.77   
...                              ...              ...              ...   
66.623306 -61.629639           58.43            51.13            46.67   
          -61.546310           57.27            48.67            44.13   
          -61.462978           57.63            50.43            46.20   
          -61.379650           58.27            51.07            46.60   
          -61.296322           60.80            53.97            50.00   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  
lat       lon                                                           
40.041039 -89.045212           90.50            76.90            74.70  
          -88.961884           90.53            76.77            74.57  
          -88.878548           90.60            76.57            74.17  
          -88.795219           89.80            75.03            72.37  
          -88.711891           89.37            74.87            71.63  
...                              ...              ...              ...  
66.623306 -61.629639           60.83            68.73            60.90  
          -61.546310           60.03            71.57            68.13  
          -61.462978           60.93            71.50            69.60  
          -61.379650           61.07            71.20            72.40  
          -61.296322           62.13            71.33            72.23  

[76206 rows x 9 columns]

# FOR RCP 85

## Subsetting arrays by periods of time and percentiles with Xclim

In [12]:
dsEns85= ens.create_ensemble(files85)
dsEns85

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    dlyfrzthw    (realization, time, lat, lon) float64 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [13]:
# Mapping data - 30 year means
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time185 = tmp185.time[0::window]

ds30yavg85 = tmp185.coarsen(time=window).mean()
ds30yavg85['time'] = time185
perc30yavg85 = ens.ensemble_percentiles(ds30yavg85)
perc30yavg85 # entire grid

<xarray.Dataset>
Dimensions:        (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * lon            (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat            (lat) float32 66.62331 66.53998 ... 40.12437 40.04104
  * time           (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * realization    (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    dlyfrzthw_p10  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    dlyfrzthw_p50  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    dlyfrzthw_p90  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [14]:
df85 = perc30yavg85.drop('realization').to_dataframe().dropna()
df85

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



dlyfrzthw_p10  dlyfrzthw_p50  dlyfrzthw_p90
lat       lon        time                                                   
66.623306 -89.045212 1981-01-01      37.700000      40.033333      43.000000
                     2011-01-01      28.900000      35.266667      39.166667
                     2041-01-01      23.166667      27.800000      37.433333
                     2071-01-01      22.366667      27.833333      48.900000
          -88.961884 1981-01-01      38.266667      40.400000      43.033333
...                                        ...            ...            ...
40.041039 -74.129143 2071-01-01      40.133333      51.266667      62.033333
          -74.045807 1981-01-01      88.166667      88.633333      92.266667
                     2011-01-01      72.066667      79.200000      83.300000
                     2041-01-01      54.766667      64.666667      70.200000
                     2071-01-01      37.000000      47.800000      58.233333

[304824 rows x 3 columns]

In [15]:
df85 = df85.reset_index()

df85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"df85_anual.feather")

In [16]:
#df85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"df85_anual.feather")

## Transform in Celsius and round

In [17]:
df85C = df85.copy()
df85C[var+"_p10"] = round(df85C[var+"_p10"], 2)
df85C[var+"_p50"] = round(df85C[var+"_p50"], 2)
df85C[var+"_p90"] = round(df85C[var+"_p90"], 2)
df85C

,lat,lon,time,dlyfrzthw_p10,dlyfrzthw_p50,dlyfrzthw_p90
0,66.623306,-89.045212,1981-01-01,37.70,40.03,43.00
1,66.623306,-89.045212,2011-01-01,28.90,35.27,39.17
2,66.623306,-89.045212,2041-01-01,23.17,27.80,37.43
3,66.623306,-89.045212,2071-01-01,22.37,27.83,48.90
4,66.623306,-88.961884,1981-01-01,38.27,40.40,43.03
...,...,...,...,...,...,...
304819,40.041039,-74.129143,2071-01-01,40.13,51.27,62.03
304820,40.041039,-74.045807,1981-01-01,88.17,88.63,92.27
304821,40.041039,-74.045807,2011-01-01,72.07,79.20,83.30
304822,40.041039,-74.045807,2041-01-01,54.77,64.67,70.20


## Eliminate 2011, Pivot Table and Create new columns 

In [19]:
Region1i85 = df85C
print (Region1i85)
Region1i285 = Region1i85[Region1i85.time.dt.year!= 2011]
print ("2011 eliminated")
df85f = Region1i285.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df85f.columns = [year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df85f

              lat        lon       time  dlyfrzthw_p10  dlyfrzthw_p50  \
0       66.623306 -89.045212 1981-01-01          37.70          40.03   
1       66.623306 -89.045212 2011-01-01          28.90          35.27   
2       66.623306 -89.045212 2041-01-01          23.17          27.80   
3       66.623306 -89.045212 2071-01-01          22.37          27.83   
4       66.623306 -88.961884 1981-01-01          38.27          40.40   
...           ...        ...        ...            ...            ...   
304819  40.041039 -74.129143 2071-01-01          40.13          51.27   
304820  40.041039 -74.045807 1981-01-01          88.17          88.63   
304821  40.041039 -74.045807 2011-01-01          72.07          79.20   
304822  40.041039 -74.045807 2041-01-01          54.77          64.67   
304823  40.041039 -74.045807 2071-01-01          37.00          47.80   

        dlyfrzthw_p90  
0               43.00  
1               39.17  
2               37.43  
3               48.90  
4  

hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212           86.97            58.60            43.47   
          -88.961884           86.90            59.03            44.03   
          -88.878548           86.67            58.87            44.37   
          -88.795219           85.60            57.87            43.77   
          -88.711891           85.10            57.93            43.73   
...                              ...              ...              ...   
66.623306 -61.629639           51.73            45.20            41.43   
          -61.546310           50.80            42.23            38.57   
          -61.462978           51.40            44.30            40.23   
          -61.379650           51.93            44.80            40.67   
          -61.296322           54.50            51.07            45.67   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212           87.33            67.53            53.70   
          -88.961884           87.50            67.90            54.57   
          -88.878548           87.60            67.67            54.63   
          -88.795219           86.57            66.93            54.27   
          -88.711891           85.83            66.53            53.97   
...                              ...              ...              ...   
66.623306 -61.629639           56.87            48.70            45.77   
          -61.546310           56.33            47.13            46.40   
          -61.462978           56.40            48.13            45.73   
          -61.379650           57.07            48.23            45.60   
          -61.296322           59.47            52.87            54.40   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212           92.70            72.47            59.87  
          -88.961884           92.60            72.53            60.23  
          -88.878548           92.53            72.07            60.20  
          -88.795219           91.53            71.17            60.00  
          -88.711891           91.10            70.73            59.63  
...                              ...              ...              ...  
66.623306 -61.629639           60.53            70.80            76.13  
          -61.546310           59.37            72.77            79.77  
          -61.462978           59.93            73.50            78.83  
          -61.379650           60.43            74.50            79.70  
          -61.296322           62.33            76.10            74.77  

[76206 rows x 9 columns]

# Merge df45 and 85

In [20]:
dftg = pd.merge(df45f, df85f, on=["lat","lon"])
dftg.head()

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212           85.50            67.47            60.50   
          -88.961884           85.67            67.73            60.93   
          -88.878548           85.40            67.63            61.03   
          -88.795219           83.73            66.67            60.40   
          -88.711891           83.20            66.17            59.67   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212           86.80            72.23            68.53   
          -88.961884           86.73            72.83            68.67   
          -88.878548           86.60            72.77            68.77   
          -88.795219           85.30            71.43            67.97   
          -88.711891           85.00            70.93            67.77   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                            
40.041039 -89.045212           90.50            76.90            74.70   
          -88.961884           90.53            76.77            74.57   
          -88.878548           90.60            76.57            74.17   
          -88.795219           89.80            75.03            72.37   
          -88.711891           89.37            74.87            71.63   

                      hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212           86.97            58.60            43.47   
          -88.961884           86.90            59.03            44.03   
          -88.878548           86.67            58.87            44.37   
          -88.795219           85.60            57.87            43.77   
          -88.711891           85.10            57.93            43.73   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212           87.33            67.53            53.70   
          -88.961884           87.50            67.90            54.57   
          -88.878548           87.60            67.67            54.63   
          -88.795219           86.57            66.93            54.27   
          -88.711891           85.83            66.53            53.97   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212           92.70            72.47            59.87  
          -88.961884           92.60            72.53            60.23  
          -88.878548           92.53            72.07            60.20  
          -88.795219           91.53            71.17            60.00  
          -88.711891           91.10            70.73            59.63

# Extract hist

In [21]:
dftg1 = dftg.drop(columns=["hist_rcp45_p10","hist_rcp45_p50", "hist_rcp45_p90", "hist_rcp85_p10", "hist_rcp85_p50", "hist_rcp85_p90"])
dftgh = dftg[["hist_rcp45_p50", "hist_rcp85_p50"]]

In [22]:
dftg1

t2050_rcp45_p10  t2080_rcp45_p10  t2050_rcp45_p50  \
lat       lon                                                             
40.041039 -89.045212            67.47            60.50            72.23   
          -88.961884            67.73            60.93            72.83   
          -88.878548            67.63            61.03            72.77   
          -88.795219            66.67            60.40            71.43   
          -88.711891            66.17            59.67            70.93   
...                               ...              ...              ...   
66.623306 -61.629639            43.70            40.43            51.13   
          -61.546310            39.97            38.63            48.67   
          -61.462978            42.30            39.43            50.43   
          -61.379650            42.77            39.90            51.07   
          -61.296322            49.37            45.23            53.97   

                      t2080_rcp45_p50  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                             
40.041039 -89.045212            68.53            76.90            74.70   
          -88.961884            68.67            76.77            74.57   
          -88.878548            68.77            76.57            74.17   
          -88.795219            67.97            75.03            72.37   
          -88.711891            67.77            74.87            71.63   
...                               ...              ...              ...   
66.623306 -61.629639            46.67            68.73            60.90   
          -61.546310            44.13            71.57            68.13   
          -61.462978            46.20            71.50            69.60   
          -61.379650            46.60            71.20            72.40   
          -61.296322            50.00            71.33            72.23   

                      t2050_rcp85_p10  t2080_rcp85_p10  t2050_rcp85_p50  \
lat       lon                                                             
40.041039 -89.045212            58.60            43.47            67.53   
          -88.961884            59.03            44.03            67.90   
          -88.878548            58.87            44.37            67.67   
          -88.795219            57.87            43.77            66.93   
          -88.711891            57.93            43.73            66.53   
...                               ...              ...              ...   
66.623306 -61.629639            45.20            41.43            48.70   
          -61.546310            42.23            38.57            47.13   
          -61.462978            44.30            40.23            48.13   
          -61.379650            44.80            40.67            48.23   
          -61.296322            51.07            45.67            52.87   

                      t2080_rcp85_p50  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                            
40.041039 -89.045212            53.70            72.47            59.87  
          -88.961884            54.57            72.53            60.23  
          -88.878548            54.63            72.07            60.20  
          -88.795219            54.27            71.17            60.00  
          -88.711891            53.97            70.73            59.63  
...                               ...              ...              ...  
66.623306 -61.629639            45.77            70.80            76.13  
          -61.546310            46.40            72.77            79.77  
          -61.462978            45.73            73.50            78.83  
          -61.379650            45.60            74.50            79.70  
          -61.296322            54.40            76.10            74.77  

[76206 rows x 12 columns]

# Merge both hist_50 and get the mean

In [23]:
dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)
dftgh2 = dftgh.reset_index()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)



In [24]:
dftgh3 = dftgh2[["lat", "lon", "hist_p50"]]
dftgh3

,lat,lon,hist_p50
0,40.041039,-89.045212,87.06
1,40.041039,-88.961884,87.12
2,40.041039,-88.878548,87.10
3,40.041039,-88.795219,85.94
4,40.041039,-88.711891,85.42
...,...,...,...
76201,66.623306,-61.629639,57.65
76202,66.623306,-61.546310,56.80
76203,66.623306,-61.462978,57.02
76204,66.623306,-61.379650,57.67


# Merge means with the 45 and 85 DF

In [25]:
dftg_all = pd.merge(dftgh3, dftg1, on=["lat","lon"])

In [26]:
dftg_all

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,87.06,67.47,60.50,72.23,68.53,76.90,74.70,58.60,43.47,67.53,53.70,72.47,59.87
1,40.041039,-88.961884,87.12,67.73,60.93,72.83,68.67,76.77,74.57,59.03,44.03,67.90,54.57,72.53,60.23
2,40.041039,-88.878548,87.10,67.63,61.03,72.77,68.77,76.57,74.17,58.87,44.37,67.67,54.63,72.07,60.20
3,40.041039,-88.795219,85.94,66.67,60.40,71.43,67.97,75.03,72.37,57.87,43.77,66.93,54.27,71.17,60.00
4,40.041039,-88.711891,85.42,66.17,59.67,70.93,67.77,74.87,71.63,57.93,43.73,66.53,53.97,70.73,59.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,57.65,43.70,40.43,51.13,46.67,68.73,60.90,45.20,41.43,48.70,45.77,70.80,76.13
76202,66.623306,-61.546310,56.80,39.97,38.63,48.67,44.13,71.57,68.13,42.23,38.57,47.13,46.40,72.77,79.77
76203,66.623306,-61.462978,57.02,42.30,39.43,50.43,46.20,71.50,69.60,44.30,40.23,48.13,45.73,73.50,78.83
76204,66.623306,-61.379650,57.67,42.77,39.90,51.07,46.60,71.20,72.40,44.80,40.67,48.23,45.60,74.50,79.70


In [27]:
dftg_all.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

In [28]:
#dftg_all = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

In [29]:
dftg_all 

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,87.06,67.47,60.50,72.23,68.53,76.90,74.70,58.60,43.47,67.53,53.70,72.47,59.87
1,40.041039,-88.961884,87.12,67.73,60.93,72.83,68.67,76.77,74.57,59.03,44.03,67.90,54.57,72.53,60.23
2,40.041039,-88.878548,87.10,67.63,61.03,72.77,68.77,76.57,74.17,58.87,44.37,67.67,54.63,72.07,60.20
3,40.041039,-88.795219,85.94,66.67,60.40,71.43,67.97,75.03,72.37,57.87,43.77,66.93,54.27,71.17,60.00
4,40.041039,-88.711891,85.42,66.17,59.67,70.93,67.77,74.87,71.63,57.93,43.73,66.53,53.97,70.73,59.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,57.65,43.70,40.43,51.13,46.67,68.73,60.90,45.20,41.43,48.70,45.77,70.80,76.13
76202,66.623306,-61.546310,56.80,39.97,38.63,48.67,44.13,71.57,68.13,42.23,38.57,47.13,46.40,72.77,79.77
76203,66.623306,-61.462978,57.02,42.30,39.43,50.43,46.20,71.50,69.60,44.30,40.23,48.13,45.73,73.50,78.83
76204,66.623306,-61.379650,57.67,42.77,39.90,51.07,46.60,71.20,72.40,44.80,40.67,48.23,45.60,74.50,79.70


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regioins: column name, short name

In [30]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
              "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"), 
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [31]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [32]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "à": "a", "è": "e", "ô" : "o", "ç":"c", "É": "E", "È": "E", "Î": "i", "Ç":"C" } 

In [33]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_annual.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_annual.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_dlyfrzthw_annual.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
